<a href="https://colab.research.google.com/github/RMBAnalytics/tcueventdashboard/blob/main/FY25Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt

# Load data
df = pd.read_excel("FY25 Event Registrants 7-7v2.xlsx", sheet_name=0)
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
df['Paid'] = df['Paid'].map({'Yes': True, 'No': False})

# Sidebar Filters
chapters = df['Chapter/Club/Group'].unique()
event_types = df['Event Type'].unique()

selected_chapters = st.sidebar.multiselect("Select Chapters/Groups", chapters, default=list(chapters))
selected_types = st.sidebar.multiselect("Select Event Types", event_types, default=list(event_types))

filtered_df = df[df['Chapter/Club/Group'].isin(selected_chapters) & df['Event Type'].isin(selected_types)]

# Summary Metrics
total_registrants = filtered_df['Registrants'].sum()
total_known = filtered_df['Known Registrants'].sum()
total_events = len(filtered_df)

st.title("FY25 Event Registration Dashboard")

col1, col2, col3 = st.columns(3)
col1.metric("Total Registrants", total_registrants)
col2.metric("Known Registrants", total_known)
col3.metric("Number of Events", total_events)

# Charts
st.subheader("Registrants by Chapter/Group")
chapter_chart = alt.Chart(
    filtered_df.groupby('Chapter/Club/Group')['Registrants'].sum().reset_index()
).mark_bar().encode(
    x=alt.X('Registrants:Q', title='Total Registrants'),
    y=alt.Y('Chapter/Club/Group:N', sort='-x', title='Chapter/Group'),
    tooltip=['Chapter/Club/Group', 'Registrants']
).properties(height=400)

st.altair_chart(chapter_chart, use_container_width=True)

st.subheader("Registrants by Event Type")
type_chart = alt.Chart(
    filtered_df.groupby('Event Type')['Registrants'].sum().reset_index()
).mark_bar().encode(
    x=alt.X('Registrants:Q', title='Total Registrants'),
    y=alt.Y('Event Type:N', sort='-x'),
    tooltip=['Event Type', 'Registrants']
).properties(height=300)

st.altair_chart(type_chart, use_container_width=True)

st.subheader("Event Table")
st.dataframe(filtered_df.sort_values(by='Event start date'))
